In [29]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
import gc
import time
import pytz
import datetime
from tqdm import tqdm
import logging
import traceback

In [30]:
df = pd.read_excel('/content/HomelyExtract_20230622.xlsx')

In [31]:
df['who_lives_here']

0        Singles,Professionals,Families With Kids,Retir...
1        Professionals,Families With Kids,Students,Beac...
2        Professionals,Families With Kids,Retirees,Beac...
3                                                      NaN
4        Families With Kids,Hipsters,Beach Lovers,Profe...
                               ...                        
11491    Families With Kids,Professionals,Singles,Retir...
11492    Families With Kids,Professionals,Singles,Retir...
11493                                                  NaN
11494             Professionals,Families With Kids,Singles
11495             Professionals,Singles,Families With Kids
Name: who_lives_here, Length: 11496, dtype: object

In [32]:
df.columns

Index(['postcode', 'state', 'suburb', 'search_string', 'ratings',
       'number_of_reviews', 'rankings', 'great_for', 'not_great_for',
       'who_lives_here', 'time_stamp', 'region', 'search_string_for_homely',
       'link_for_homely'],
      dtype='object')

In [33]:
df['great_for']

0        Schools,Shopping options,Gym and fitness,Inter...
1        Childcare,Clean & green,Gym and fitness,Intern...
2        Clean & green,Gym and fitness,Internet access,...
3                                                      NaN
4        Clean & green,Medical facilities,Parks and rec...
                               ...                        
11491    Neighbourly spirit,Safe and sound,Parks and re...
11492    Clean & green,Cost of living,Eating out,Intern...
11493                                                  NaN
11494    Clean & green,Neighbourly spirit,Parks and rec...
11495    Clean & green,Neighbourly spirit,Parks and rec...
Name: great_for, Length: 11496, dtype: object

In [34]:
df['great_for'] = df['great_for'].str.replace('Clean & greenSafe and soundParks and recreation', 'Clean & green,Safe and sound,Parks and recreation')

df['great_for'] = df['great_for'].fillna('')

df_great_for = df['great_for'].str.split(',').apply(lambda x: pd.Series(1, [word.strip().lower().replace(' ', '_') for word in x])).fillna(0)

# Add prefix to the new columns
df_great_for = df_great_for.add_prefix('great_for_')
df_great_for.columns

Index(['great_for_schools', 'great_for_shopping_options',
       'great_for_gym_and_fitness', 'great_for_internet_access',
       'great_for_parks_and_recreation', 'great_for_childcare',
       'great_for_clean_&_green', 'great_for_medical_facilities',
       'great_for_lack_of_traffic', 'great_for_', 'great_for_parking',
       'great_for_public_transport', 'great_for_pest-free',
       'great_for_peace_and_quiet', 'great_for_resale_or_rental_value',
       'great_for_neighbourly_spirit', 'great_for_eating_out',
       'great_for_cost_of_living', 'great_for_safe_and_sound',
       'great_for_nightlife', 'great_for_families_with_kids',
       'great_for_retirees', 'great_for_tourists', 'great_for_country_lovers',
       'great_for_professionals', 'great_for_singles', 'great_for_hipsters',
       'great_for_students', 'great_for_trendy_&_stylish',
       'great_for_beach_lovers', 'great_for_lgbt+'],
      dtype='object')

In [35]:
counts = df_great_for.iloc[:, :-1].sum().sort_values(ascending=False)
print(counts)

great_for_                          6875.0
great_for_clean_&_green             2656.0
great_for_parks_and_recreation      2506.0
great_for_neighbourly_spirit        2337.0
great_for_safe_and_sound            1973.0
great_for_internet_access           1696.0
great_for_peace_and_quiet           1418.0
great_for_schools                   1247.0
great_for_parking                   1182.0
great_for_public_transport          1160.0
great_for_cost_of_living             884.0
great_for_lack_of_traffic            871.0
great_for_childcare                  871.0
great_for_eating_out                 836.0
great_for_resale_or_rental_value     688.0
great_for_gym_and_fitness            659.0
great_for_medical_facilities         633.0
great_for_pest-free                  323.0
great_for_shopping_options           267.0
great_for_nightlife                   72.0
great_for_families_with_kids           5.0
great_for_retirees                     4.0
great_for_professionals                4.0
great_for_t

In [36]:
columns_to_drop = ['great_for_families_with_kids','great_for_retirees','great_for_professionals','great_for_tourists','great_for_country_lovers','great_for_singles','great_for_hipsters','great_for_students','great_for_trendy_&_stylish','great_for_beach_lovers']
df_great_for.drop(columns=columns_to_drop, inplace=True)

In [37]:
len(df_great_for.columns)

21

In [38]:
df = pd.concat([df, df_great_for], axis=1)

In [39]:
del df['great_for_']

In [40]:
df.columns

Index(['postcode', 'state', 'suburb', 'search_string', 'ratings',
       'number_of_reviews', 'rankings', 'great_for', 'not_great_for',
       'who_lives_here', 'time_stamp', 'region', 'search_string_for_homely',
       'link_for_homely', 'great_for_schools', 'great_for_shopping_options',
       'great_for_gym_and_fitness', 'great_for_internet_access',
       'great_for_parks_and_recreation', 'great_for_childcare',
       'great_for_clean_&_green', 'great_for_medical_facilities',
       'great_for_lack_of_traffic', 'great_for_parking',
       'great_for_public_transport', 'great_for_pest-free',
       'great_for_peace_and_quiet', 'great_for_resale_or_rental_value',
       'great_for_neighbourly_spirit', 'great_for_eating_out',
       'great_for_cost_of_living', 'great_for_safe_and_sound',
       'great_for_nightlife', 'great_for_lgbt+'],
      dtype='object')

In [41]:
df['not_great_for'] = df['not_great_for'].fillna('')

df_not_great_for = df['not_great_for'].str.split(',').apply(lambda x: pd.Series(1, [word.strip().lower().replace(' ', '_') for word in x])).fillna(0)

# Add prefix to the new columns
df_not_great_for = df_not_great_for.add_prefix('not_great_for_')
df_not_great_for.columns

Index(['not_great_for_', 'not_great_for_nightlife', 'not_great_for_parking',
       'not_great_for_cost_of_living', 'not_great_for_lack_of_traffic',
       'not_great_for_eating_out', 'not_great_for_gym_and_fitness',
       'not_great_for_parks_and_recreation', 'not_great_for_public_transport',
       'not_great_for_shopping_options',
       'not_great_for_resale_or_rental_value',
       'not_great_for_medical_facilities', 'not_great_for_internet_access',
       'not_great_for_schools', 'not_great_for_pest-free',
       'not_great_for_clean_&_green', 'not_great_for_safe_and_sound',
       'not_great_for_peace_and_quiet', 'not_great_for_childcare',
       'not_great_for_neighbourly_spirit', 'not_great_for_professionals',
       'not_great_for_families_with_kids', 'not_great_for_tourists',
       'not_great_for_hipsters', 'not_great_for_trendy_&_stylish',
       'not_great_for_beach_lovers', 'not_great_for_singles',
       'not_great_for_retirees', 'not_great_for_country_lovers',
       

In [42]:
counts = df_not_great_for.iloc[:, :-1].sum().sort_values(ascending=False)
print(counts)

not_great_for_                          9111.0
not_great_for_nightlife                 1734.0
not_great_for_public_transport           623.0
not_great_for_shopping_options           614.0
not_great_for_eating_out                 568.0
not_great_for_gym_and_fitness            523.0
not_great_for_childcare                  405.0
not_great_for_medical_facilities         390.0
not_great_for_parking                    302.0
not_great_for_cost_of_living             210.0
not_great_for_pest-free                  185.0
not_great_for_lack_of_traffic            136.0
not_great_for_internet_access            103.0
not_great_for_schools                     80.0
not_great_for_resale_or_rental_value      76.0
not_great_for_clean_&_green               50.0
not_great_for_parks_and_recreation        50.0
not_great_for_peace_and_quiet             47.0
not_great_for_neighbourly_spirit          36.0
not_great_for_safe_and_sound              34.0
not_great_for_professionals                4.0
not_great_for

In [43]:
columns_to_drop = ['not_great_for_professionals','not_great_for_families_with_kids','not_great_for_tourists','not_great_for_trendy_&_stylish','not_great_for_singles','not_great_for_retirees','not_great_for_hipsters','not_great_for_beach_lovers','not_great_for_country_lovers','not_great_for_lgbt+']
df_not_great_for.drop(columns=columns_to_drop, inplace=True)

In [44]:
len(df_not_great_for.columns)

21

In [45]:
df = pd.concat([df, df_not_great_for], axis=1)

In [46]:
df[df['not_great_for_']==1]['not_great_for'].head()

0    
1    
2    
3    
5    
Name: not_great_for, dtype: object

In [47]:
del df['not_great_for_']

In [48]:
df['who_lives_here'] = df['who_lives_here'].fillna('')

df_who_lives_here = df['who_lives_here'].str.split(',').apply(lambda x: pd.Series(1, [word.strip().lower().replace(' ', '_') for word in x])).fillna(0)

# Add prefix to the new columns
df_who_lives_here = df_who_lives_here.add_prefix('who_lives_here_')

In [49]:
len(df_who_lives_here.columns)

56

In [50]:
counts = df_who_lives_here.iloc[:, :-1].sum().sort_values(ascending=False)

# Print the counts
print(counts)

who_lives_here_                                                                                                             6768.0
who_lives_here_families_with_kids                                                                                           4585.0
who_lives_here_professionals                                                                                                4094.0
who_lives_here_retirees                                                                                                     3784.0
who_lives_here_singles                                                                                                      3356.0
who_lives_here_country_lovers                                                                                                762.0
who_lives_here_tourists                                                                                                      740.0
who_lives_here_lgbtq+                                                              

In [51]:
df_who_lives_here = df_who_lives_here[['who_lives_here_families_with_kids','who_lives_here_professionals','who_lives_here_retirees','who_lives_here_singles','who_lives_here_country_lovers','who_lives_here_tourists','who_lives_here_lgbtq+','who_lives_here_students','who_lives_here_fashion_conscious','who_lives_here_beach_lovers','who_lives_here_hipsters']]
df_who_lives_here

,who_lives_here_families_with_kids,who_lives_here_professionals,who_lives_here_retirees,who_lives_here_singles,who_lives_here_country_lovers,who_lives_here_tourists,who_lives_here_lgbtq+,who_lives_here_students,who_lives_here_fashion_conscious,who_lives_here_beach_lovers,who_lives_here_hipsters
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
11491,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11492,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11494,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
len(df_who_lives_here.columns)

11

In [52]:
df = pd.concat([df, df_who_lives_here], axis=1)

In [61]:
df.columns

Index(['postcode', 'state', 'suburb', 'search_string', 'ratings',
       'number_of_reviews', 'rankings', 'great_for', 'not_great_for',
       'who_lives_here', 'time_stamp', 'region', 'search_string_for_homely',
       'link_for_homely', 'great_for_schools', 'great_for_shopping_options',
       'great_for_gym_and_fitness', 'great_for_internet_access',
       'great_for_parks_and_recreation', 'great_for_childcare',
       'great_for_clean_&_green', 'great_for_medical_facilities',
       'great_for_lack_of_traffic', 'great_for_parking',
       'great_for_public_transport', 'great_for_pest-free',
       'great_for_peace_and_quiet', 'great_for_resale_or_rental_value',
       'great_for_neighbourly_spirit', 'great_for_eating_out',
       'great_for_cost_of_living', 'great_for_safe_and_sound',
       'great_for_nightlife', 'great_for_lgbt+', 'not_great_for_nightlife',
       'not_great_for_parking', 'not_great_for_cost_of_living',
       'not_great_for_lack_of_traffic', 'not_great_for_eatin

In [63]:
df.iloc[:, 14:] = df.iloc[:, 14:].astype(int)

<ipython-input-63-7250dda2210e>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 14:] = df.iloc[:, 14:].astype(int)


In [67]:
df['postcode'] = df['postcode'].apply(lambda x: f"0{x}" if len(str(x)) == 3 else str(x))
df.head()

,postcode,state,suburb,search_string,ratings,number_of_reviews,rankings,great_for,not_great_for,who_lives_here,...,who_lives_here_professionals,who_lives_here_retirees,who_lives_here_singles,who_lives_here_country_lovers,who_lives_here_tourists,who_lives_here_lgbtq+,who_lives_here_students,who_lives_here_fashion_conscious,who_lives_here_beach_lovers,who_lives_here_hipsters
0,0800,Northern Territory,Darwin City,"Darwin City, NT 0800",4.0,2,Ranked 13th best suburb by locals in Northern ...,"Schools,Shopping options,Gym and fitness,Inter...",,"Singles,Professionals,Families With Kids,Retir...",...,1,1,1,0,0,0,0,0,0,1
1,0810,Northern Territory,Alawa,"Alawa, NT 0810",4.6,0,NaN,"Childcare,Clean & green,Gym and fitness,Intern...",,"Professionals,Families With Kids,Students,Beac...",...,1,0,0,0,0,0,1,0,1,0
2,0810,Northern Territory,Brinkin,"Brinkin, NT 0810",4.6,1,NaN,"Clean & green,Gym and fitness,Internet access,...",,"Professionals,Families With Kids,Retirees,Beac...",...,1,1,0,0,0,0,0,0,1,0
3,0810,Northern Territory,Casuarina,"Casuarina, NT 0810",0.0,0,NaN,,,,...,0,0,0,0,0,0,0,0,0,0
4,0810,Northern Territory,Coconut Grove,"Coconut Grove, NT 0810",4.7,3,NaN,"Clean & green,Medical facilities,Parks and rec...",Nightlife,"Families With Kids,Hipsters,Beach Lovers,Profe...",...,1,0,1,0,0,0,0,0,1,1


In [72]:
len(df.columns)

65

In [71]:
for i in df.columns:
  print(i)

postcode
state
suburb
search_string
ratings
number_of_reviews
rankings
great_for
not_great_for
who_lives_here
time_stamp
region
search_string_for_homely
link_for_homely
great_for_schools
great_for_shopping_options
great_for_gym_and_fitness
great_for_internet_access
great_for_parks_and_recreation
great_for_childcare
great_for_clean_&_green
great_for_medical_facilities
great_for_lack_of_traffic
great_for_parking
great_for_public_transport
great_for_pest-free
great_for_peace_and_quiet
great_for_resale_or_rental_value
great_for_neighbourly_spirit
great_for_eating_out
great_for_cost_of_living
great_for_safe_and_sound
great_for_nightlife
great_for_lgbt+
not_great_for_nightlife
not_great_for_parking
not_great_for_cost_of_living
not_great_for_lack_of_traffic
not_great_for_eating_out
not_great_for_gym_and_fitness
not_great_for_parks_and_recreation
not_great_for_public_transport
not_great_for_shopping_options
not_great_for_resale_or_rental_value
not_great_for_medical_facilities
not_great_for_int

In [68]:
df.to_excel('HomelyExtract_20230622.xlsx', index = False)